# Importation des librairies python

In [ ]:
import sqla from sqlalchemy
import time
import 

# Création des différentes classes du logiciel

# Création de la classe réservation

## Pour la class reservation :
##### On peut supposer pour simplifier que les emails ont une structure ressemblant aux exemple cidessous, c’est-à-dire que chaque demande de la réservation se présente de la façon suivante : 
##### mention de la salle, puis de la date (jour avant mois), puis mention du créneau horaire.
##### Mail1.txt :
##### Bonjour,
##### Nous souhaiterions utiliser la salle 736 le mardi 17 janvier entre 15 h et 19 h et la salle 140 le 
##### 18 janvier 9 h – 11 h.
##### Mail2.txt :
##### Salut,
##### J’ai besoin de la salle 736 le 17 janvier entre 8 h et 10 h.
##### Mail3.txt :
##### Je peux avoir la salle 736 le mardi 17 janvier de 14 h à 16 h ?
##### Chris.
##### L’objet reservation doit pouvoir doit pouvoir contenir toutes ces informations, stockée sous forme de 
##### chaine de caractères, sauf les horaires qui seront des entiers pour pouvoir effectuer plus facilement 
##### des opérations

In [1]:
class reservation:
    def __init__(self, addresse_email, contenu_mail, nomSalle, dateReserv, heureReserve):
        

SyntaxError: unexpected EOF while parsing (<ipython-input-1-3a2844108409>, line 1)

### Création de la classe interface_mail

In [ ]:
class interface_mail:
    

### Création de la classe Data access object

In [ ]:
class DAO:

### Création de la classe moteur

In [ ]:
class moteur: